In [1]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))

os.chdir("../")

# How does coarsening work

In [14]:
import xarray as xr
import zarr
import numcodecs
import numpy as np
import glob
import torch
import era5_data_proc
import os

RAW_ERA5_PATH = "/vol/bitbucket/bet20/dataset/era5/global_full"

In [16]:
nc_files = glob.glob(f'{RAW_ERA5_PATH}/2022*.nc')
nc_files.sort()

data = xr.open_dataset(nc_files[0], chunks={"time": 4})
data

<xarray.Dataset> Size: 49GB
Dimensions:    (longitude: 1440, latitude: 721, level: 8, time: 124)
Coordinates:
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * level      (level) int32 32B 50 150 250 400 500 600 850 1000
  * time       (time) datetime64[ns] 992B 2022-01-01 ... 2022-01-31T18:00:00
Data variables:
    z          (time, level, latitude, longitude) float64 8GB dask.array<chunksize=(4, 8, 721, 1440), meta=np.ndarray>
    q          (time, level, latitude, longitude) float64 8GB dask.array<chunksize=(4, 8, 721, 1440), meta=np.ndarray>
    t          (time, level, latitude, longitude) float64 8GB dask.array<chunksize=(4, 8, 721, 1440), meta=np.ndarray>
    u          (time, level, latitude, longitude) float64 8GB dask.array<chunksize=(4, 8, 721, 1440), meta=np.ndarray>
    v          (time, level, latitude, longitude) float64 8GB dask.array<chunksize=(4, 8, 721, 1440), meta=np.ndarray>
    w          (time, level, latitude, longitude) float64 8GB dask.array<chunksize=(4, 8, 721, 1440), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-26 11:59:48 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [17]:
coarsened_data = data.coarsen(longitude=22, latitude=22, boundary="pad").mean()

/vol/bitbucket/bet20/nlam/lib/python3.9/site-packages/xarray/core/rolling.py:1217: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explicitly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  reduced[key] = da.variable.coarsen(
/vol/bitbucket/bet20/nlam/lib/python3.9/site-packages/xarray/core/rolling.py:1217: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the 

In [18]:
coarsened_data

<xarray.Dataset> Size: 104MB
Dimensions:    (time: 124, level: 8, latitude: 33, longitude: 66)
Coordinates:
  * longitude  (longitude) float32 264B 2.625 8.125 13.62 ... 349.1 354.6 358.6
  * latitude   (latitude) float32 132B 87.38 81.88 76.38 ... -77.62 -83.12 -88.0
  * level      (level) int32 32B 50 150 250 400 500 600 850 1000
  * time       (time) datetime64[ns] 992B 2022-01-01 ... 2022-01-31T18:00:00
Data variables:
    z          (time, level, latitude, longitude) float64 17MB dask.array<chunksize=(4, 8, 32, 65), meta=np.ndarray>
    q          (time, level, latitude, longitude) float64 17MB dask.array<chunksize=(4, 8, 32, 65), meta=np.ndarray>
    t          (time, level, latitude, longitude) float64 17MB dask.array<chunksize=(4, 8, 32, 65), meta=np.ndarray>
    u          (time, level, latitude, longitude) float64 17MB dask.array<chunksize=(4, 8, 32, 65), meta=np.ndarray>
    v          (time, level, latitude, longitude) float64 17MB dask.array<chunksize=(4, 8, 32, 65), meta=np.ndarray>
    w          (time, level, latitude, longitude) float64 17MB dask.array<chunksize=(4, 8, 32, 65), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-26 11:59:48 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [6]:
coarsened_data.to_netcdf("test.nc")

In [7]:
cc = xr.open_dataset("test.nc")

In [10]:
dataset = "era5_global"
path = f"data/{dataset}/samples/train/20220101000000.npy"

data = np.load(path)

In [12]:
data.shape

(2178, 48)

In [20]:
era5_data_proc.uk_subset(data)

<xarray.Dataset> Size: 176MB
Dimensions:    (time: 124, level: 8, latitude: 65, longitude: 57)
Coordinates:
  * longitude  (longitude) float32 228B 350.0 350.2 350.5 350.8 ... 3.5 3.75 4.0
  * latitude   (latitude) float32 260B 63.0 62.75 62.5 62.25 ... 47.5 47.25 47.0
  * level      (level) int32 32B 50 150 250 400 500 600 850 1000
  * time       (time) datetime64[ns] 992B 2022-01-01 ... 2022-01-31T18:00:00
Data variables:
    z          (time, level, latitude, longitude) float64 29MB dask.array<chunksize=(4, 8, 65, 40), meta=np.ndarray>
    q          (time, level, latitude, longitude) float64 29MB dask.array<chunksize=(4, 8, 65, 40), meta=np.ndarray>
    t          (time, level, latitude, longitude) float64 29MB dask.array<chunksize=(4, 8, 65, 40), meta=np.ndarray>
    u          (time, level, latitude, longitude) float64 29MB dask.array<chunksize=(4, 8, 65, 40), meta=np.ndarray>
    v          (time, level, latitude, longitude) float64 29MB dask.array<chunksize=(4, 8, 65, 40), meta=np.ndarray>
    w          (time, level, latitude, longitude) float64 29MB dask.array<chunksize=(4, 8, 65, 40), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-26 11:59:48 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...